In [ ]:
user_input = input("Putanja do fajla: ")
file_path = user_input

In [65]:

with open(file_path, 'r') as file:
    lines = file.readlines()

n = int(lines[0])
distances = []
times = []

for i in range(1, n + 2):
    split = lines[i].split(" ")
    distances.append([int(x) for x in split[:-1]])

for i in range(n + 2, 2 * n + 2):
    split = lines[i].split(" ")
    times.append([int(x) for x in split[:-1]])

In [ ]:
import random
import numpy as np
import copy

class TSPAutomaton:
    def __init__(self, n=None):
        self.route = []
        self.order = []
        if n is not None:
            self.init(n)

    def init(self, n):
        self.route = [0] + random.sample(range(1, n), n-1) + [0]
        self.order = random.sample(range(1, len(times)+1), len(times))

    def distance(self):
        total_distance = sum(distances[self.route[i]][self.route[i + 1]] for i in range(len(self.route) - 1))
        return total_distance

    def time(self):
        total_time = 0
        route = self.route[1:-1]
        for i in range(len(self.order)-1):
            if i>len(self.order) or i<0:
                continue
            total_time+= times[route[i]-1][self.order[i]-1]
        return total_time

    def fitness(self):
        total_cost = self.distance() + self.time()
        return 1 / total_cost

    def mutate(self):
        mutation_type = random.choice(['swap', 'reverse', 'insert'])
        route_middle = self.route[1:-1]  # Exclude the starting and ending 0
        order_middle = self.order
        if mutation_type == 'swap':
            i, j = random.sample(range(len(route_middle)), 2)
            route_middle[i], route_middle[j] = route_middle[j], route_middle[i]
            i, j = random.sample(range(len(order_middle)), 2)
            order_middle[i], order_middle[j] = order_middle[j], order_middle[i]
        elif mutation_type == 'reverse':
            i, j = sorted(random.sample(range(len(route_middle)), 2))
            route_middle[i:j + 1] = list(reversed(route_middle[i:j + 1]))
            i, j = sorted(random.sample(range(len(order_middle)), 2))
            order_middle[i:j + 1] = list(reversed(order_middle[i:j + 1]))
        elif mutation_type == 'insert':
            i, j = random.sample(range(len(route_middle)), 2)
            route_middle.insert(i, route_middle.pop(j))
            i, j = random.sample(range(len(order_middle)), 2)
            order_middle.insert(i, order_middle.pop(j))
        self.route = [0] + route_middle + [0]
        self.order = order_middle

    def replicate(self):
        new_automaton = TSPAutomaton()
        new_automaton.route = copy.deepcopy(self.route)
        new_automaton.order = copy.deepcopy(self.order)
        new_automaton.mutate()
        return new_automaton

def tournament(pop, size):
    select = random.sample(pop, size)
    return max(select, key=lambda x: x[1])

# Algorithm parameters
pop_size = 20
city_count = len(distances)
pop = [TSPAutomaton(city_count) for _ in range(pop_size)]
pop = sorted([(a, a.fitness()) for a in pop], key=lambda x: -x[1])

for i in range(1, 4):
    print("#", i)
    iters = 500

    while pop[0][1] < 1 and iters:
        iters -= 1
        npop = pop[:]
        for a_old in pop:
            a = a_old[0].replicate()
            npop.append((a, a.fitness()))
        pop = []
        for i in range(pop_size):
            a = tournament(npop, 3)
            while a in pop:
                a = tournament(npop, 3)
            pop.append(a)
        pop.sort(key=lambda x: -x[1])
        # if iters % 10 == 0:
        #     print('Current best fitness:', pop[0][1], ' iters left:', iters)

    best_route = pop[0][0].route
    best_order = pop[0][0].order
    print('Best route:', best_route)
    print('Best order:', best_order)
    print('Total distance:', 1 / pop[0][1])
